In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
df = pd.read_csv('../data/data_clean.csv')

df.head(1)

,City,State,Bank,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,SBA_Appv,Default,NAICS_i,isNewBusiness,isFranchise,SBARatio,InterestRate
0,EVANSVILLE,IN,FIFTH THIRD BANK,1997,84,4,0,0,1,0,0,1,60000.0,48000.0,0,45,1.0,0,0.8,5.19


In [4]:
# Select Features (X) and Target (Y)
X = df[['City','State','Bank','ApprovalFY','Term','NoEmp','CreateJob','RetainedJob','FranchiseCode','UrbanRural','RevLineCr','LowDoc','GrAppv','SBA_Appv','NAICS_i','isNewBusiness','isFranchise','SBARatio','InterestRate']]
Y = df['Default']

In [5]:
# Do a train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
